# Fine Tune Com Wikipédia



### Descriçao

---
1. O objetivo deste notebook é aprender sobre Fine Tune
2. Vamos descorrer sobre modelos de LLM e utilizar a biblioteca Torch para descorrer sobre assunto
3. Vamos ensinar nossa máquina com os dados do [Wikipédia](https://pt.wikipedia.org/wiki/Ci%C3%AAncia)
---

### Baixando os pacotes




In [ ]:
!pip install transformers transformers[torch] datasets

### Documentação

1. **Transformers**:
   - [Documentação do Hugging Face Transformers](https://huggingface.co/transformers/)
   - [Repositório GitHub](https://github.com/huggingface/transformers)

2. **Requests**:
   - [Documentação do Requests](https://docs.python-requests.org/en/latest/)
   - [Repositório GitHub](https://github.com/psf/requests)

3. **Datasets**:
   - [Documentação do Hugging Face Datasets](https://huggingface.co/docs/datasets/)
   - [Repositório GitHub](https://github.com/huggingface/datasets)

4. **Torch**:
   - [Documentação do PyTorch](https://pytorch.org/docs/stable/index.html)
   - [Repositório GitHub](https://github.com/pytorch/pytorch)


### Importando as bibliotecas

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import requests
from datasets import Dataset, load_dataset
import torch

### importando o modelo

In [ ]:
model_name = "pierreguillou/gpt2-small-portuguese"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Requisitando o Wikipedia

In [ ]:


url = "https://pt.wikipedia.org/wiki/Ci%C3%AAncia"
response = requests.get(url)
text_data = response.text

lines = text_data.split('\n')
data_dict = {"text": lines}

dataset = Dataset.from_dict(data_dict)

In [ ]:
# tokenização do dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
first_example = tokenized_dataset[1250]
print(first_example)
print(first_example['text'])

In [ ]:
data_collator = DataCollatorForLanguageModeling( tokenizer=tokenizer,mlm=False)

In [ ]:
# hiper parâmetros
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1, # !!!
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2
)

In [ ]:
trainer = Trainer(model=model,args=training_args, data_collator=data_collator, train_dataset=tokenized_dataset)

In [ ]:
trainer.train()

### Salvando o modelo

In [ ]:
model.save_pretrained("meu_modelo_treinado")
tokenizer.save_pretrained("meu_modelo_treinado")

# Utilizando o Modelo

#importando a biblioteca que criamos

In [ ]:
from transformers import pipeline

In [ ]:
gerador = pipeline("text-generation", model="meu_modelo_treinado")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=60, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
print(resultado)